In [2]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [4]:
def marching_cube(thres, cells):
	# vertices use dictionary to avoid duplicate axes
	vertex_array = {}
	face_array = []
	t1 = time.time()
	# -------------------TODO------------------
	# compute vertices and faces
	# vertices: [N, 3]
	# faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
	# for-loop is allowed to reduce difficulty
	N1, N2, N3 = cells.shape
	vertex_index = 0
	for x in range(N1 - 1):
		for y in range(N2 - 1):
			for z in range(N3 - 1):
				vertex_for_face = []
				case_num = getCaseNum(x, y, z, thres, cells)
				
				for case_value in case_num:
					if case_value == -1:
						continue
					c1 = np.array(
						[
							x + CaseNum2EdgeOffset[case_value][0],
							y + CaseNum2EdgeOffset[case_value][1],
							z + CaseNum2EdgeOffset[case_value][2],
						]
					)
					c2 = np.array(
						[
							x + CaseNum2EdgeOffset[case_value][3],
							y + CaseNum2EdgeOffset[case_value][4],
							z + CaseNum2EdgeOffset[case_value][5],
						]
					)

					value_c1 = cells[c1[0], c1[1], c1[2]]
					value_c2 = cells[c2[0], c2[1], c2[2]]

					alpha_1 = abs(thres - value_c2) / (
						abs(thres - value_c1) + abs(thres - value_c2)
					)
					# adjust the decimal point to reduce the repeated vertex
					vertex = tuple(np.round(alpha_1 * c1 + (1 - alpha_1) * c2, 3))

					if vertex not in vertex_array:
						vertex_array[vertex] = vertex_index
						vertex_index += 1
						
					vertex_for_face.append(vertex_array[vertex])
					if len(vertex_for_face) == 3:
						face_array.append(vertex_for_face)
						vertex_for_face = []
	# -------------------TODO------------------
	t2 = time.time()
	print("\nTime taken by algorithm\n" + "-" * 40 + "\n{} s".format(t2 - t1))
	vertex_array = list(vertex_array.keys())	# keys: vertex coordinates, values: vertex index
	return np.array(vertex_array), np.array(face_array)

In [5]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
	data = np.load(os.path.join('data', shape_name + '_cell.npy'))

	verts, faces = marching_cube(0, data)
	print(f"shape_name: {shape_name}")
	print(f"verts shape: {verts.shape}")
	print(f"faces shape: {faces.shape}")
	
	mesh = trimesh.Trimesh(vertices=verts, faces=faces)
	mesh_txt = trimesh.exchange.obj.export_obj(mesh)
	with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
		fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
0.46604418754577637 s
shape_name: spot
verts shape: (6850, 3)
faces shape: (13712, 3)

Time taken by algorithm
----------------------------------------
0.4857621192932129 s
shape_name: bob
verts shape: (8959, 3)
faces shape: (17936, 3)


The results visulized by MeshLab are shown below. 
<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="../results/bob_visual.png" alt="Uniform Sampling" style="width: 100%;">
    </div>
    <div>
        <img src="../results/spot_visual.png" alt="FPS Sampling" style="width: 100%;">
    </div>
</div>